# Exploring Raw Data

Here are just some very simple examples of going through and inspecting the raw data, and making some plots using `ctapipe`.
The data explored here are *raw Monte Carlo* data, which is Data Level "R0" in CTA terminology (e.g. it is before any processing that would happen inside a Camera or off-line)

Setup:

In [ ]:
from ctapipe.utils import get_dataset_path
from ctapipe.io import event_source, EventSeeker
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from matplotlib import pyplot as plt
from astropy import units as u
%matplotlib inline

To read SimTelArray format data, ctapipe uses the `pyeventio` library (which is installed automatically along with ctapipe). The following lines however will load any data known to ctapipe (multiple `EventSources` are implemented, and chosen automatically based on the type of the input file. 

All data access first starts with an `EventSource`, and here we use a helper function `event_source` that constructs one. The resulting `source` object can be iterated over like a list of events.  We also here use an `EventSeeker` which provides random-access to the source (by seeking to the given event ID or number)

In [ ]:
source = event_source(get_dataset_path("gamma_test_large.simtel.gz"), max_events=100)
seeker = EventSeeker(source)

## Explore the contents of an event

note that the R0 level is the raw data that comes out of a camera, and also the lowest level of monte-carlo data. 

In [ ]:
event = seeker[0]  # get first event
event

the event is just a class with a bunch of data items in it.  You can see a more compact represntation via:

In [ ]:
print(repr(event.r0))

printing the event structure, will currently print the value all items under it (so you get a lot of output if you print a high-level container):

In [ ]:
print(event.mc)

In [ ]:
print(event.r0.tels_with_data)

note that the event has 2 telescopes in it: 38,40... Let's try the next one:

In [ ]:
event = seeker[1] # get the next event
print(event.r0.tels_with_data)

now, we have a larger event with many telescopes... Let's look at the data from **CT7**:

In [ ]:
teldata = event.r0.tel[7]
print(teldata)
teldata

Note that some values are unit quantities (`astropy.units.Quantity`) or angular quantities (`astropy.coordinates.Angle`), and you can easily maniuplate them:

In [ ]:
event.mc.energy

In [ ]:
event.mc.energy.to('GeV')

In [ ]:
event.mc.energy.to('J')

In [ ]:
event.mc.alt

In [ ]:
print("Altitude in degrees:", event.mc.alt.deg)

## Look for signal pixels in a camera
again, `event.r0.tel[x]` contains a data structure for the telescope data, with some fields like `waveform`.

Let's make a 2D plot of the sample data (sample vs pixel), so we can see if we see which pixels contain Cherenkov light signals:

In [ ]:
plt.pcolormesh(teldata.waveform[0])  # note the [0] is for channel 0
plt.colorbar()
plt.xlabel("sample number")
plt.ylabel("Pixel_id")

Let's zoom in to see if we can identify the pixels that have the Cherenkov signal in them

In [ ]:
plt.pcolormesh(teldata.waveform[0])
plt.colorbar()
plt.ylim(260,290)
plt.xlabel("sample number")
plt.ylabel("pixel_id")
print("waveform[0] is an array of shape (N_pix,N_slice) =",teldata.waveform[0].shape)

Now we can really see that some pixels have a signal in them!

Lets look at a 1D plot of pixel 270 in channel 0 and see the signal:

In [ ]:
trace = teldata.waveform[0][270]   
plt.plot(trace, ls='steps')

Great! It looks like a *standard Cherenkov signal*!

Let's take a look at several traces to see if the peaks area aligned:

In [ ]:
for pix_id in [269,270,271,272,273,274,275,276]:
    plt.plot(teldata.waveform[0][pix_id], label="pix {}".format(pix_id), ls='steps')
plt.legend()

## Look at the time trace from a Camera Pixel

`ctapipe.calib.camera` includes classes for doing automatic trace integration with many methods, but before using that, let's just try to do something simple!

Let's define the integration windows first:
By eye, they seem to be reaonsable from sample 8 to 13 for signal, and 20 to 29 for pedestal (which we define as the sum of all noise: NSB + electronic)

In [ ]:
for pix_id in [269,270,271,272,273,274,275,276]:
    plt.plot(teldata.waveform[0][pix_id],'+-')
plt.fill_betweenx([0,1200],20,29,color='red',alpha=0.3, label='Ped window')
plt.fill_betweenx([0,1200],8,13,color='green',alpha=0.3, label='Signal window')
plt.legend()

## Do a very simplisitic trace analysis 
Now, let's for example calculate a signal and background in a the fixed windows we defined for this single event.  Note we are ignoring the fact that cameras have 2 gains, and just using a single gain (channel 0, which is the high-gain channel):

In [ ]:
data = teldata.waveform[0]
peds = data[:, 20:29].mean(axis=1)  # mean of samples 20 to 29 for all pixels
sums = data[:, 8:13].sum(axis=1)/(13-8)    # simple sum integration

In [ ]:
phist = plt.hist(peds, bins=50, range=[0,150])
plt.title("Pedestal Distribution of all pixels for a single event")

let's now take a look at the pedestal-subtracted sums and a pedestal-subtracted signal:


In [ ]:
plt.plot(sums - peds)
plt.xlabel("pixel id")
plt.ylabel("Pedestal-subtracted Signal")

Now, we can clearly see that the signal is centered at 0 where there is no Cherenkov light, and we can also clearly see the shower around pixel 250.

In [ ]:
# we can also subtract the pedestals from the traces themselves, which would be needed to compare peaks properly
for ii in range(270,280):
    plt.plot(data[ii] - peds[ii], ls='steps', label="pix{}".format(ii))
plt.legend()

## Camera Displays

It's of course much easier to see the signal if we plot it in 2D with correct pixel positions! 

>note: the instrument data model is not fully implemented, so there is not a good way to load all the camera information (right now it is hacked into the `inst` sub-container that is read from the Monte-Carlo file)

In [ ]:
camgeom = event.inst.subarray.tel[24].camera

In [ ]:
title="CT24, run {} event {} ped-sub".format(event.r0.obs_id,event.r0.event_id)
disp = CameraDisplay(camgeom,title=title)
disp.image = sums - peds 
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)  # autoscale

It looks like a nice signal! We have plotted our pedestal-subtracted trace integral, and see the shower clearly!

Let's look at all telescopes:

> note we plot here the raw signal, since we have not calculated the pedestals for each)

In [ ]:
for tel in event.r0.tels_with_data:
    plt.figure()
    camgeom = event.inst.subarray.tel[tel].camera
    title="CT{}, run {} event {}".format(tel,event.r0.obs_id,event.r0.event_id)
    disp = CameraDisplay(camgeom,title=title)
    disp.image = event.r0.tel[tel].waveform[0].sum(axis=1)
    disp.cmap = plt.cm.RdBu_r
    disp.add_colorbar()
    disp.set_limits_percent(95)

## some signal processing...

Let's try to detect the peak using the scipy.signal package:
http://docs.scipy.org/doc/scipy/reference/signal.html

In [ ]:
from scipy import signal
import numpy as np

In [ ]:
pix_ids = np.arange(len(data))
has_signal = sums > 300

widths = np.array([8,]) # peak widths to search for (let's fix it at 8 samples, about the width of the peak)
peaks = [signal.find_peaks_cwt(trace,widths) for trace in data[has_signal] ]

for p,s in zip(pix_ids[has_signal],peaks):
    print("pix{} has peaks at sample {}".format(p,s))
    plt.plot(data[p], ls='steps-mid')
    plt.scatter(np.array(s),data[p,s])

clearly the signal needs to be filtered first, or an appropriate wavelet used, but the idea is nice